# 创建错误数据

In [2]:
import pandas as pd
import numpy as np
import os,random

# os.chdir(r"E:\git\data-science\pandas_demo\data")

# 获得数据（可用相对路径获取文件）
datas = pd.read_csv('data/titanic_train.csv')

In [ ]:
# 明确缺失值、重复值、异常值
na_value = np.nan
duplicate_vlaues = []
exception_values = [99999999,0.0000001,'卯月']

# 找到我们待会儿要使用的随机索引
rows = random.sample(range(891),50)
columns = random.sample(range(12),6)

# step1：获得我们的重复值
for _ in range(10):
  i = random.choice(rows)
  duplicate_value = datas.iloc[i,:]
  duplicate_vlaues.append(duplicate_value)

duplicate_vlaues = pd.DataFrame(duplicate_vlaues)

# 覆盖原数据并且重置索引
datas = pd.concat((datas,duplicate_vlaues),axis=0,ignore_index=True)

# step2 ：设定我们的缺失值
for _ in range(50):
  i = random.choice(rows)
  j = random.choice(columns)
  datas.iloc[i,j] = np.nan

# step3 : 设定我们的异常值
for _ in range(50):
  i = random.choice(rows)
  j = random.choice(columns)
  exception_value = random.choice(exception_values)
  datas.iloc[i,j] = exception_value

# 将我们修改后的数据保存到一个新的csv文件中
datas.to_csv('titanic_train_matt.csv')

In [ ]:
# 使用info属性查看信息
datas = pd.read_csv('titanic_train_matt.csv',index_col=0)
datas.info

In [ ]:
# 查看各列缺失值的比例
# 首先根据isnull()方法获得我们的是否是缺失值的901行12列的布尔数组
# 之后再针对布尔数组True=1，False=0的性质求平均值，得到各列缺失值所占的比例。
datas.isnull().mean()

In [ ]:
# 查看有多少重复行
datas.duplicated().sum()

# 有哪些重复行数据
datas.loc[datas.duplicated(),:]

In [ ]:
# 查看数值型数据的异常值（已知异常值的情况）
datas.describe([.01,.10,.25,.75,.99]).T

# 查看object对象的异常值
datas.describe(include='object').T

# 数据预览

pandas_profiling依赖包过多且有冲突

AttributeError: module 'numba' has no attribute 'generated_jit'

In [ ]:
import pandas as pd
import numpy as np
import os,pandas_profiling

pandas_profiling.ProfileReport(datas)

# 处理重复值

In [ ]:
# 删除完全重复的记录
datas = pd.read_csv('titanic_train_matt.csv',index_col=0)
datas_pd = datas.copy()
display(datas_pd.shape)
datas_pd.drop_duplicates(inplace=True,ignore_index=True)
display(datas_pd.shape)
display(datas_pd.info)
datas_pd.to_csv('titanic_train_matt_duplicated.csv')

# 异常值处理

In [ ]:
# 首先我们查看有哪些异常值
datas = pd.read_csv('titanic_train_matt.csv',index_col=0)
datas_pd = datas.copy()
datas.describe([.01,.25,.50,.75,.99]).T

In [ ]:
# 查看一下object对象有哪些缺失值
datas_pd.describe(include='object').T

In [ ]:
# 船费‘Fare’明显出现了极大的异常值，同时我们的船上的隔间‘Cabin’出现了‘卯月’明显不对。
# 对船费分析，船费的高低和Pclass（船舱等级）有关，因此可以根据船舱等级求得平均数或者众数作为异常值的替换。
# 而对于隔间而言，能否存活和隔间的关系不大，更何况隔间缺失严重，高达76%，因此可以直接丢弃。
datas_pd.drop('Cabin',axis=1,inplace=True)
datas_pd.shape

In [ ]:
# 填补船费
# 首先根据‘Pclass’查看一下各个等级船舱的平均值
datas_pd.groupby(by='Pclass').mean()

In [23]:
# 查看数据后发现，卯月的中位数和2.0的中位数接近，99999999的中位数和2.0的中位数接近，1e-07的中位数和2.0的中位数接近。
# 因此，我们将‘Pclass’的三个异常值全部判定为2.0，在填充之前我们再看一些各个‘Pclass’的类型
datas_pd.loc[:,'Pclass'].unique()

array([3, 1, 2], dtype=int64)

In [24]:
# 异常值处理
datas_pd.loc[datas_pd.loc[:,'Pclass']=='1e-07','Pclass']='2.0'
datas_pd.loc[datas_pd.loc[:,'Pclass']=='99999999','Pclass']='2.0'
datas_pd.loc[datas_pd.loc[:,'Pclass']=='卯月','Pclass']='2.0'

# 查看是否还有异常值
datas_pd.groupby(by='Pclass').median()

C:\Users\wangkan\AppData\Local\Temp\ipykernel_12816\65388791.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  datas_pd.loc[datas_pd.loc[:,'Pclass']=='1e-07','Pclass']='2.0'


TypeError: agg function failed [how->median,dtype->object]